# Experiments

In [4]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import AffinityClustering as ACC
from Plotter import Plotter
from DataReader import DataReader
from DataModifier import DataModifier

## Preparation

In [ ]:
from datetime import datetime

start_time = datetime.now()
print('Starting time:', start_time)

file_location = 'Results_buckets/'
plotter = Plotter(None, None, file_location)

## Read data

In [ ]:
# Read data
from sklearn.datasets import make_circles
from sklearn.preprocessing import StandardScaler
# data_reader = DataReader()
# loc = 'datasets/sklearn/data_two_circles.csv'
# V, n, E = data_reader.read_data_set_from_csvfile(loc)
# print('Read dataset: ', loc)


X, y = make_circles()

X = StandardScaler().fit_transform(X)




In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from mstfordensegraphs import partion_vertices, get_edges, find_mst

conf = SparkConf().setAppName('MST_algo')
sc = SparkContext.getOrCreate(conf=conf)

n = len(X)
c = 0
epsilon = 0

k = math.ceil(n ** ((c - epsilon) / 2))
print("k: ", k)

U, V = partion_vertices(V, k)

rddUV = sc.parallelize(U)\
    .cartesian(sc.parallelize(V))\
        .map(lambda x: get_edges(x[0], x[1], E))\
            .map(lambda x: (find_mst(x[0], x[1]), x[2]))

both = rddUV.collect()


ImportError: cannot import name 'SparkConf' from 'pyspark.sql' (/Users/glovali/repos/MSTforDenseGraphs/mpa_ml/lib/python3.13/site-packages/pyspark/sql/__init__.py)